# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
mongo.list_database_names()

['admin',
 'classDB',
 'config',
 'dbname',
 'epa',
 'fruits_db',
 'gardenDB',
 'local',
 'met',
 'petsitly_marketing',
 'travel_db',
 'uk_food']

In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'Walmer And Kingsdown Golf Club',
 'AddressLine2': 'The Leas',
 'AddressLine3': 'Kingsdown',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Halfway Hut',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.792282474894,
 'FHRSID': 1531040,
 'LocalAuthorityBusinessID': 'PI/000085904',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT14 8EP',
 'RatingDate': '1901-01-01T00:00:00',
 'RatingKey': 'fhrs_awaitinginspection_en-gb',
 'RatingValue': 'AwaitingInspection',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65983d0019aebd564991ad23'),
 'geocode': {'latitude': '51.1787284', 'longitude': '1.4032986'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1531040',
            'rel': 'self'}],
 'meta':

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [23]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

DuplicateKeyError: E11000 duplicate key error collection: uk_food.establishments index: _id_ dup key: { _id: ObjectId('6598480f76984ff3ed3f843e') }, full error: {'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: uk_food.establishments index: _id_ dup key: { _id: ObjectId('6598480f76984ff3ed3f843e') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('6598480f76984ff3ed3f843e')}}

In [11]:
# Check that the new restaurant was inserted
pprint(db.establishments.find_one({
    'BusinessName': 'Penang Flavours'
    })
)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65983ebeb790658d1a020287'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [51]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

# Define query
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}

# Define fields to include in results. Exclude object id.
projection = {'BusinessTypeID': 1, 'BusinessType': 1, '_id': 0}

# Run query and store in variable
results = db.establishments.find(query, projection)

# store returned BusinessTypeID as a variable for future use
biztype = results[0]['BusinessTypeID']

# log statement
print(f'''
First result is: {results[0]}
Storing "{biztype}" as BusinessTypeID
''')


First result is: {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
Storing "1" as BusinessTypeID



3. Update the new restaurant with the `BusinessTypeID` you found.

In [46]:
# Update the new restaurant with the correct BusinessTypeID
query = {'BusinessName': 'Penang Flavours'}
new_val = new_value = {'$set': {'BusinessTypeID': biztype}}

db.establishments.update_one(query, new_val)

In [52]:
# Confirm that the new restaurant was updated
pprint(db.establishments.find_one({
    'BusinessName': 'Penang Flavours'
    })
)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65983ebeb790658d1a020287'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [53]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = db.establishments.count_documents({'LocalAuthorityName': 'Dover'})
print("Number of documents with LocalAuthorityName as 'Dover':", dover_count)


Number of documents with LocalAuthorityName as 'Dover': 994


In [54]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db.establishments.delete_many({'LocalAuthorityName': 'Dover'})

# Output the count of documents deleted
print("Number of documents deleted:", delete_result.deleted_count)


Number of documents deleted: 994


In [55]:
# Check if any remaining documents include Dover
dover_count = db.establishments.count_documents({'LocalAuthorityName': 'Dover'})
print("Number of documents with LocalAuthorityName as 'Dover':", dover_count)

Number of documents with LocalAuthorityName as 'Dover': 0


In [56]:
# Check that other documents remain with 'find_one'
db.establishments.find_one()

{'_id': ObjectId('65983d0019aebd564991b008'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [57]:
# Change the data type from String to Decimal for longitude and latitude
result = db.establishments.update_many(
    {},  
    [
        {
            '$set': {
                'geocode.longitude': {
                    '$toDecimal': '$geocode.longitude'
                },
                'geocode.latitude': {
                    '$toDecimal': '$geocode.latitude'
                }
            }
        }
    ]
)

print("Number of documents updated:", result.modified_count)

Number of documents updated: 38788


Use `update_many` to convert `RatingValue` to integer numbers.

In [58]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [60]:
# Change the data type from String to Integer for RatingValue
result = db.establishments.update_many(
    {},
    [
        {
            '$set': {
                'RatingValue': {
                    '$toInt': '$RatingValue'
                }
            }
        }
    ]
)

print("Number of documents updated:", result.modified_count)

Number of documents updated: 0


In [67]:
# Check that the coordinates and rating value are now numbers

total_documents = db.establishments.count_documents({})

# Check if 'RatingValue' is now an integer
rating_value_int_count = db.establishments.count_documents({'RatingValue': {'$type': 'int'}})

# Check if 'geocode.longitude' and 'geocode.latitude' are now Decimal128
longitude_decimal_count = db.establishments.count_documents({'geocode.longitude': {'$type': 'decimal'}})
latitude_decimal_count = db.establishments.count_documents({'geocode.latitude': {'$type': 'decimal'}})

print("Total documents:", total_documents)
print(f"Documents where 'RatingValue' is an integer: {rating_value_int_count} (diff:{total_documents - rating_value_int_count})")
print(f"Documents where 'geocode.longitude' is Decimal128: {longitude_decimal_count}")
print(f"Documents where 'geocode.latitude' is Decimal128: {latitude_decimal_count}")


Total documents: 38788
Documents where 'RatingValue' is an integer: 34694 (diff:4094)
Documents where 'geocode.longitude' is Decimal128: 38788
Documents where 'geocode.latitude' is Decimal128: 38788


In [62]:
null_or_missing_rating_count = db.establishments.count_documents({
    '$or': [
        {'RatingValue': {'$exists': False}},  # RatingValue does not exist
        {'RatingValue': None}  # RatingValue is explicitly set to null
    ]
})

print("Documents with null or missing 'RatingValue':", null_or_missing_rating_count)



Documents with null or missing 'RatingValue': 4094
